In [1]:
import EOBRun_module
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.constants as const
import astropy.constants as aconst
from pycbc.waveform import td_approximants, fd_approximants, get_td_waveform
from pycbc.detector import Detector
from pycbc.filter import match, optimized_match
from pycbc.psd import aLIGOZeroDetHighPower
from pycbc.types import timeseries

In [4]:
# Global settings
sample_rate = 4096

# Useful functions

## Miscellaneous

In [40]:
# Finds next highest power of two
def ceiltwo(number):
    ceil = math.ceil(np.log2(number))
    return 2**ceil

## Generating waveforms

In [42]:
# Generates EccentricTD waveform with given parameters
def gen_e_td_wf(M, q, e, f):
    m2 = M / (1+q)
    m1 = M - m2
    e_td_p, e_td_c = get_td_waveform(approximant='EccentricTD',
                                     mass1=m1,
                                     mass2=m2,
                                     eccentricity=e,
                                     delta_t=1.0/sample_rate,
                                     f_lower=f)
    return e_td_p, e_td_c

## Combining waveforms

In [43]:
# Resize both waveform lengths to next highest power of two
def resize_wfs(wf1, wf2):
    tlen = ceiltwo(max(len(wf1), len(wf2)))
    wf1.resize(tlen)
    wf2.resize(tlen)
    return wf1, wf2

In [ ]:
# Gets overall waveform h = A*h1 + B*h2
def get_h(A, B):

    # Gets h_ap and h_peri components which make up overall waveform
    h_ap = get_h_ap()
    h_peri = get_h_peri()

    # Edits sample times of h_peri to match h_ap
    first_ind = np.argmin(np.abs(h_ap.sample_times[0]-h_peri.sample_times))
    h_peri = h_peri[first_ind:]
    h_ap, h_peri = resize_wfs(h_ap, h_peri)
    h_peri.start_time = h_ap.start_time
    assert np.array_equal(h_ap.sample_times, h_peri.sample_times)

    # Calculate h1, h2 components of waveform
    h1 = 0.5*(h_ap + h_peri)
    h2 = 0.5*(h_ap - h_peri)

    # Returns overall waveform using complex coefficients A, B, as well as componen
    return A*h1 + B*h2, h1, h2